# RNN 원리
## Numpy로 RNN 원리 구현하기

In [1]:
import numpy as np

timesteps = 10  # 시점의 개수. - NLP 에서는 보통 문장의 길이
input_dim = 4  # 입력의 차원. 단어 벡터의 차원

hidden_size = 8  # 은닉 유닛의 개수 (메모리 셀의 용량이라고도 한다.)

# 입력 데이터 배열 만들기
inputs = np.random.random((timesteps, input_dim))  # 입력에 해당되는 2D 텐서(임베딩 벡터)  # 미리 준비된 가중치

# 은닉 상태의 크기 hidden_size로 셀의 유닛을 만든다.
hidden_state_t = np.zeros((hidden_size,))

print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [2]:
# 여기까지가 초기화 작업

In [3]:
# 현 시점 입력을 위한 가중치
Wx = np.random.random((hidden_size, input_dim))  # WX + B 할것이므로 8*4

# 이전 시점에 대한 hidden state도 받아야 함
Wh = np.random.random((hidden_size, hidden_size))  # 8개가 나와서 8개로 다시 들어가니까 8*8

# 편향은 unit 당 1개씩
b = np.random.random((hidden_size,))

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [5]:
# 메모리 셀 작동시키기
total_hidden_states = []

# 각 시점에서의 입력값 가져오기
for input_t in inputs:
  # Wx * Xt + Wh * Ht-1 + b
  # 에 tanh를 함
  output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
  
  total_hidden_states.append(list(output_t))  # 각 시점의 은닉 상태의 값을 계속 축적
  print(np.shape(total_hidden_states))

  hidden_state_t = output_t  # 현재의 output을 다음 시점의 입력으로 삼기 위해 hidden_state_t에 대입

total_hidden_states = np.stack(total_hidden_states, axis=0)
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.852453   0.88327393 0.94810212 0.88941995 0.93481959 0.58532311
  0.89431276 0.946098  ]
 [0.99998446 0.99985832 0.99999367 0.99993066 0.99996999 0.99997249
  0.99996003 0.99997499]
 [0.99999547 0.99993738 0.99999779 0.99997154 0.9999873  0.99999746
  0.99999533 0.99999408]
 [0.99999089 0.99993224 0.99999615 0.99996184 0.99997394 0.99999494
  0.99997984 0.99998684]
 [0.99999351 0.99997468 0.99999859 0.99998012 0.99998366 0.99999435
  0.99997961 0.99999518]
 [0.99999645 0.99997085 0.99999844 0.99998022 0.99998508 0.99999067
  0.99998315 0.99999525]
 [0.9999944  0.99997489 0.99999834 0.99997914 0.9999834  0.9999917
  0.99997146 0.99999417]
 [0.99999497 0.99991621 0.99999663 0.99996488 0.99997862 0.99999502
  0.99999161 0.99999063]
 [0.99999355 0.99995736 0.99999722 0.99997028 0.9999821  0.99999382
  0.99997677 0.99999031]
 [0.99999015 0.99996281 0.99999727 0.99996918 0.99998411 0.99999634
  0.99997336 0.99998943]]

# Tensorflow로 Vanilla RNN 구현

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))  # 3 -> hidden_state 갯수
                                             # input_shape -> input_length = 2, input_dim = 10
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________
